In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Origin.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Deformula.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SparseMatrix.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/NMarkov.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SymbolicDiff.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SymbolicMarkov.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/DD.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/FaultTree.jl.git"))

In [1]:
using SparseArrays
using NMarkov
using SymbolicDiff
using SymbolicMarkov
using Plots
using Distributions
using Random
using FaultTree

#### Original SHARPE code

```
* aircraft flight
* control system

ftree Non_Computers
basic IRS exp (mIRS)
kofn I2of3 2, 3, IRS IRS IRS
basic PRS exp (mPRS)
kofn P2of3 2, 3, PRS PRS PRS
basic SA exp (mSA)
kofn S2of3 2, 3, SA SA SA
or TOP I2of3 P2of3 S2of3
end

ftree CRASH
basic CS exp (mCS)
kofn C3of4 3, 4, CS CS CS CS
basic others cdf (Non_Computers)
or TOP C3of4 others
end

bind
mIRS .000015
mPRS .000019
mSA .000037
mCS .00048
end

*expr value (10;CRASH)
```

In [2]:
@ftree Non_Computers(t, mIRS, mPRS, mSA) begin
    @basic begin
        IRS = 1 - exp(-mIRS*t)
        PRS = 1 - exp(-mPRS*t)
        SA = 1 - exp(-mSA*t)
    end
    I2of3 = kofn(2, IRS, IRS, IRS)
    P2of3 = kofn(2, PRS, PRS, PRS)
    S2of3 = kofn(2, SA, SA, SA)
    TOP = I2of3 | P2of3 | S2of3
end

@ftree CRASH(t, mCS, Non_Computers) begin
    @basic begin
        CS = 1 - exp(-mCS * t)
        others = prob(Non_Computers)
    end
    C3of4 = kofn(3, CS, CS, CS, CS)
    TOP = C3of4 | others
end

CRASH (generic function with 1 method)

In [3]:
env = SymbolicEnv()
@bind env begin
    mIRS = 0.000015
    mPRS = 0.000019
    mSA = 0.000037
    mCS = 0.00048
end

(:mIRS, :mPRS, :mSA, :mCS)

In [4]:
t = 10
s = prob(CRASH(t, mCS, Non_Computers(t, mIRS, mPRS, mSA)))

"node28"

In [5]:
seval(s, env)

1.023813664844782e-6

In [6]:
[seval(s, :mIRS, env), seval(s, :mPRS, env), seval(s, :mSA, env), seval(s, :mCS, env)] # first derivative

4-element Vector{Float64}:
 0.008996617037420041
 0.01139457587070375
 0.022179461016862533
 0.002725275798442721

In [7]:
seval(s, (:mIRS, :mPRS), env)

-0.00010251274036652392

### Original SHARPE code

##### avail.txt
```
format 8

include param_sharpe
include models.txt

bind
	gamma			gamma_hang + gamma_die
	pxy_gamma		pxy_gamma_hang + pxy_gamma_die
end

expr sysprob(CLUSTER)
expr sysprob(CM)
expr sysprob(BLADE)
expr 1-exrss(nic)

end
```

##### param_sharpe
```
bind	b_os	0.95
bind	rou_a	41.0390478956483
bind	delta_m	2
bind	pxy_rou_a	76.1791903844933
bind	rou_m	10.9849424929091
bind	mu_esw	2
bind	phi	1241.37931034483
bind	mo_new	3
bind	mu_ps	2
bind	pxy_rou_m	12.5323316751202
bind	n_i	3
bind	delta_os	2
bind	lambda_esw	4.07871429136622e-007
bind	pxy_beta_m	5.38628248508112
bind	mu_base	2
bind	n_o	4
bind	mu_nic	2
bind	pxy_delta1	3600
bind	pxy_delta2	5.24781341107872
bind	lambda_c	2.0547943798086e-007
bind	mu_mp	2
bind	lambda_nic	6.43835819290738e-007
bind	gamma_die	0.000342465753424658
bind	m_stable1	4
bind	m_stable2	4
bind	m_setup	4
bind	beta_m	5.40946656649136
bind	pxy_gamma_die	0.000342465753424658
bind	beta_os	6
bind	lambda_os	0.000114155251141553
bind	lambda_hd	1.2214605736956e-006
bind	pxy_gamma_hang	0.000342465753424658
bind	mu_c	2
bind	delta1	356.282784139751
bind	delta2	5.24781341107872
bind	lambda	100
bind	mu_mem	2
bind	mu_2c	1.71428571428571
bind	t_setup	0.00416666666666667
bind	lambda_base	3.49314470351934e-006
bind	lambda_mem	1.12442935210381e-006
bind	mu_2hd	1.71428571428571
bind	pxy_delta_m	2
bind	gamma_hang	0.000342465753424658
bind	pxy_phi	360
bind	pxy_mu	0.25
bind	lambda_ps	2.46575072246496e-006
bind	chi_hd	12
bind	rw_i	0.00111111111111111
bind	mu_cpu	2
bind	b	0.95
bind	c	0.95
bind	rw_o	0.00333333333333333
bind	d	0.95
bind	e	0.95
bind	c_ps	0.99
bind	pxy_b	0.95
bind	pxy_c	0.95
bind	lambda_cpu	8.10502162825022e-007
bind	mu_2ps	1.71428571428571
bind	pxy_d	0.95
bind	pxy_e	0.95
bind	t_stable1	0.0125
bind	t_stable2	0.0166666666666667
bind	mu_os	2
bind	q	0.95
bind	alpha_sp	0.4
bind	mi_new	2
bind	r	0.95
bind	mu_hd	2
bind	lambda_mp	1.60958799376124e-006
bind	mu	0.25
bind	pxy_q	0.95
bind	pxy_r	0.95
bind	c_mp	0.0036101083032491
```

##### models.txt
```
*========availability model for replication domain, Figure 1========*
markov pairup
2U UO   2*gamma
UO 1D d*delta1
UO 2D e*delta2
UO 1N (1-d)*delta1
1N 2D e*delta2
1N UN (1-e)*delta2
UO 2N (1-e)*delta2
2N UN (1-d)*delta1
2N 1D2N d*delta1
1D FS c*phi
1D FN (1-c)*phi
FS UA e*delta2
FS UR (1-e)*delta2
FN UC e*delta2
FN US (1-e)*delta2
2D 1D2D delta1
1D2D UA c*phi
1D2D UC (1-c)*phi
UA UR (1-q)*rou_a
UA 2U q*rou_a
UR 2U r*rou_m
UR UB (1-r)*rou_m
UB 2U beta_m
UC US (1-q)*rou_a
UC 2U q*rou_a
US 2U r*rou_m
US UT (1-r)*rou_m
UT 2U b*beta_m
UT RP (1-b)*beta_m
RP 2U mu
UN MD delta_m
MD 1D2N delta1
1D2N UR c*phi
1D2N US (1-c)*phi
end
  2U 1
end

*========availability model for proxy servers, Figure 8========*
markov pxy_pairup
2U UO   2 * pxy_gamma
UO 1D pxy_d * pxy_delta1
UO 2D pxy_e * pxy_delta2
UO 1N (1-pxy_d)*pxy_delta1
1N 2D pxy_e*pxy_delta2
1N UN (1-pxy_e)*pxy_delta2
UO 2N (1-pxy_e)*pxy_delta2
2N UN (1-pxy_d)*pxy_delta1
2N 1D2N pxy_d*pxy_delta1
1D SO pxy_c * pxy_phi
1D SN (1-pxy_c) * pxy_phi
SO UA pxy_e * pxy_delta2
SO UR (1-pxy_e) * pxy_delta2
SN UC pxy_e * pxy_delta2
SN US (1-pxy_e) * pxy_delta2
2D 1D2D pxy_delta1
1D2D UA pxy_c * pxy_phi
1D2D UC (1-pxy_c) * pxy_phi
UA UR (1-pxy_q) * pxy_rou_a
UA 2U pxy_q * pxy_rou_a
UR 2U pxy_r * pxy_rou_m
UR UB (1-pxy_r) * pxy_rou_m
UB 2U pxy_beta_m
UC US (1-pxy_q) * pxy_rou_a
UC 2U pxy_q * pxy_rou_a
US 2U pxy_r * pxy_rou_m
US UT (1-pxy_r) * pxy_rou_m
UT 2U pxy_b * pxy_beta_m
UT RP (1-pxy_b) * pxy_beta_m
RP 2U pxy_mu
UN MD pxy_delta_m 
MD 1D2N pxy_delta1
1D2N UR	pxy_c * pxy_phi
1D2N US	(1-pxy_c) * pxy_phi
end
  2U 1
end

*========availability model for single appserver, Figure 19========*
markov appserver
UP UO 	gamma
UO UA	e*delta2
UO 1D	d*delta1
UO 1N 	(1-d)*delta1
UO 2N	(1-e)*delta2
1N UA	e*delta2
1N UN	(1-e)*delta2
2N UR	d*delta1
2N UN	(1-d)*delta1
1D UA	e*delta2
1D UR	(1-e)*delta2
UA UR	(1-q)*rou_a
UR UB	(1-r)*rou_m
UB RE	(1-b)*beta_m
UN UR	delta_m
UA UP	q*rou_a
UR UP 	r*rou_m
UB UP	b*beta_m
RE UP	mu
reward
  UP 1
end
end

*========availability model for single proxy server, Figure 19========*
markov proxy
UP UO 	pxy_gamma
UO UA	pxy_e*pxy_delta2
UO 1N 	(1-pxy_d)*pxy_delta1
UO 2N	(1-pxy_e)*pxy_delta2
1N UA	pxy_e*pxy_delta2
1N UN	(1-pxy_e)*pxy_delta2
2N UR	pxy_d*pxy_delta1
2N UN	(1-pxy_d)*pxy_delta1
UA UR	(1-pxy_q)*pxy_rou_a
UR UB	(1-pxy_r)*pxy_rou_m
UB RE	(1-pxy_b)*pxy_beta_m
UO 1D	pxy_d*pxy_delta1
1D UA	pxy_e*pxy_delta2
1D UR	(1-pxy_e)*pxy_delta2
UN UR	pxy_delta_m
UA UP	pxy_q*pxy_rou_a
UR UP 	pxy_r*pxy_rou_m
UB UP	pxy_b*pxy_beta_m
RE UP	pxy_mu
reward
  UP 1
end
end


*=====================availability models for hardware=============================
*=======Figure 13==========*
markov midplane
UP DN 	c_mp*lambda_mp
UP U1 	(1-c_mp)*lambda_mp
U1 RP 	alpha_sp
RP UP	mu_mp
DN RP	alpha_sp
reward
  UP 1
  U1 1  		
end
end

*=======Figure 14==========*
markov cooling
UP U1 	2*lambda_c
U1 RP 	alpha_sp
U1 DN	lambda_c
DN DW	alpha_sp
RP UP	mu_c
RP DW	lambda_c
DW UP	mu_2c
reward
  UP 1
  U1 1
  RP 1
end
end

*=======Figure 15==========*
markov power
UP U1 	2*c_ps*lambda_ps
UP DN	2*(1-c_ps)*lambda_ps
U1 RP 	alpha_sp
U1 DN	lambda_ps
DN DW	alpha_sp
RP UP	mu_ps
RP DW	lambda_ps
DW UP	mu_2ps
reward
  UP 1
  U1 1
  RP 1
end
end

*=======Figure 16==========*
markov processor
UP D1 	2*lambda_cpu
D1 RP	alpha_sp
RP UP	mu_cpu
reward
  UP 1
end
end

*=======Figure 17==========*
markov memory
UP D1 	4*lambda_mem
D1 RP	alpha_sp
RP UP	mu_mem
reward
  UP 1
end
end

*=======Figure 18==========*
markov disk
UP U1	2*lambda_hd
U1 RP	alpha_sp
RP CP	mu_hd
CP DW	lambda_hd
U1 DN	lambda_hd
DN DW	alpha_sp
CP UP 	chi_hd
DW UP	mu_2hd
reward
  UP 1
  U1 1
  CP 1
end
end

*=======Figure 20==========*
markov base
UP DN lambda_base
DN RP alpha_sp
RP UP mu_base
reward 
  UP 1
end
end

*=======Figure 21==========*
markov OS
UP DN lambda_os
DN DT delta_os
DT UP b_os*beta_os
DT DW (1-b_os)*beta_os
DW RP alpha_sp
RP UP mu_os
reward 
  UP 1
end
end

*=======Figure 20==========*
markov nic
UP DN lambda_nic
DN RP alpha_sp
RP UP mu_nic
reward 
  UP 1
end
end

*=======Figure 20==========*
markov switch
UP DN lambda_esw
DN RP alpha_sp
RP UP mu_esw
reward 
  UP 1
end
end

bind	
	f_esw 		prob(switch, UP)*lambda_esw
	f_nic 		prob(nic, UP)*lambda_nic
	u_esw 		f_esw/(1-exrss(switch))
	u_nic 		f_nic/(1-exrss(nic))
end	
*=======Figure 22==========*
ftree network
repeat nic1 	exp(lambda_nic)
repeat nic2 	exp(lambda_nic)
repeat esw1		exp(lambda_esw)
repeat esw2		exp(lambda_esw)
or eth1 nic1 esw1
or eth2 nic2 esw2
and top eth1 eth2
end

*=======Figure 11==========*
ftree CM
repeat MP prob(1-exrss(midplane))
repeat Cool prob(1-exrss(cooling))
repeat Pwr  prob(1-exrss(power))
or top MP Cool Pwr
end

*=======part of Figure 12==========*
ftree BLADE
repeat Base prob(1-exrss(base))
repeat CPU prob(1-exrss(processor))
repeat Mem prob(1-exrss(memory))
repeat RAID prob(1-exrss(disk))
repeat OS prob(1-exrss(OS))
or top Base CPU Mem RAID OS
end
*=========================================


*========system availability model=========*
*=======Figure 10==========*
ftree CLUSTER
repeat CM1  prob(sysprob(CM))
repeat CM2  prob(sysprob(CM))
repeat esw1 prob(1-exrss(switch))
repeat esw2 prob(1-exrss(switch))
repeat esw3 prob(1-exrss(switch))
repeat esw4 prob(1-exrss(switch))
basic SW prob(1-exrss(appserver))
basic SWP prob(1-exrss(proxy))

	*===========Blade Servers===========*
repeat Node_A prob(sysprob(BLADE))
repeat nic1_A prob(1-exrss(nic))
repeat nic2_A prob(1-exrss(nic))
or eth1_A nic1_A esw1
or eth2_A nic2_A esw2
and eth_A eth1_A eth2_A
or BS_A Node_A eth_A

repeat Node_B prob(sysprob(BLADE))
repeat nic1_B prob(1-exrss(nic))
repeat nic2_B prob(1-exrss(nic))
or eth1_B nic1_B esw1
or eth2_B nic2_B esw2
and eth_B eth1_B eth2_B
or BS_B Node_B eth_B

repeat Node_C prob(sysprob(BLADE))
repeat nic1_C prob(1-exrss(nic))
repeat nic2_C prob(1-exrss(nic))
or eth1_C nic1_C esw1
or eth2_C nic2_C esw2
and eth_C eth1_C eth2_C
or BS_C Node_C eth_C

repeat Node_D prob(sysprob(BLADE))
repeat nic1_D prob(1-exrss(nic))
repeat nic2_D prob(1-exrss(nic))
or eth1_D nic1_D esw3
or eth2_D nic2_D esw4
and eth_D eth1_D eth2_D
or BS_D Node_D eth_D

repeat Node_E prob(sysprob(BLADE))
repeat nic1_E prob(1-exrss(nic))
repeat nic2_E prob(1-exrss(nic))
or eth1_E nic1_E esw3
or eth2_E nic2_E esw4
and eth_E eth1_E eth2_E
or BS_E Node_E eth_E

repeat Node_F prob(sysprob(BLADE))
repeat nic1_F prob(1-exrss(nic))
repeat nic2_F prob(1-exrss(nic))
or eth1_F nic1_F esw3
or eth2_F nic2_F esw4
and eth_F eth1_F eth2_F
or BS_F Node_F eth_F

repeat Node_G prob(sysprob(BLADE))
repeat nic1_G prob(1-exrss(nic))
repeat nic2_G prob(1-exrss(nic))
or eth1_G nic1_G esw1
or eth2_G nic2_G esw2
and eth_G eth1_G eth2_G
or BS_G Node_G eth_G

repeat Node_H prob(sysprob(BLADE))
repeat nic1_H prob(1-exrss(nic))
repeat nic2_H prob(1-exrss(nic))
or eth1_H nic1_H esw3
or eth2_H nic2_H esw4
and eth_H eth1_H eth2_H
or BS_H Node_H eth_H

	*===========Application Servers===========*
or AS1 SW BS_A CM1
or AS2 SW BS_A CM1
or AS3 SW BS_B CM1
or AS4 SW BS_B CM1
or AS5 SW BS_C CM1
or AS6 SW BS_C CM1
or AS7 SW BS_D CM2
or AS8 SW BS_D CM2
or AS9 SW BS_E CM2
or AS10 SW BS_E CM2
or AS11 SW BS_F CM2
or AS12 SW BS_F CM2
kofn apps 6, 12, AS1 AS2 AS3 AS4 AS5 AS6 AS7 AS8 AS9 AS10 AS11 AS12

	*===========Proxy Servers===========*
or PX1 SWP BS_G CM1
or PX2 SWP BS_H CM2
and pxys PX1 PX2

or top apps pxys
end
```


In [8]:
## *========availability model for replication domain, Figure 1========*
@markov pairup(gamma, d, delta1, e, delta2, c, phi, q, rou_a, r, rou_m, b, beta_m, mu, delta_m) begin
    @tr begin
        @s("2U") => @s("UO"), 2*gamma
        @s("UO") => @s("1D"), d*delta1
        @s("UO") => @s("2D"), e*delta2
        @s("UO") => @s("1N"), (1-d)*delta1
        @s("1N") => @s("2D"), e*delta2
        @s("1N") => @s("UN"), (1-e)*delta2
        @s("UO") => @s("2N"), (1-e)*delta2
        @s("2N") => @s("UN"), (1-d)*delta1
        @s("2N") => @s("1D2N"), d*delta1
        @s("1D") => @s("FS"), c*phi
        @s("1D") => @s("FN"), (1-c)*phi
        @s("FS") => @s("UA"), e*delta2
        @s("FS") => @s("UR"), (1-e)*delta2
        @s("FN") => @s("UC"), e*delta2
        @s("FN") => @s("US"), (1-e)*delta2
        @s("2D") => @s("1D2D"), delta1
        @s("1D2D") => @s("UA"), c*phi
        @s("1D2D") => @s("UC"), (1-c)*phi
        @s("UA") => @s("UR"), (1-q)*rou_a
        @s("UA") => @s("2U"), q*rou_a
        @s("UR") => @s("2U"), r*rou_m
        @s("UR") => @s("UB"), (1-r)*rou_m
        @s("UB") => @s("2U"), beta_m
        @s("UC") => @s("US"), (1-q)*rou_a
        @s("UC") => @s("2U"), q*rou_a
        @s("US") => @s("2U"), r*rou_m
        @s("US") => @s("UT"), (1-r)*rou_m
        @s("UT") => @s("2U"), b*beta_m
        @s("UT") => @s("RP"), (1-b)*beta_m
        @s("RP") => @s("2U"), mu
        @s("UN") => @s("MD"), delta_m
        @s("MD") => @s("1D2N"), delta1
        @s("1D2N") => @s("UR"), c*phi
        @s("1D2N") => @s("US"), (1-c)*phi
    end

    @init begin
        @s("2U"), 1
    end
end

pairup (generic function with 1 method)

In [9]:
### *========availability model for proxy servers, Figure 8========*
@markov pxy_pairup(pxy_gamma, pxy_d, pxy_delta1, pxy_e, pxy_delta2, pxy_c, pxy_phi, pxy_q, pxy_rou_a, pxy_r, pxy_rou_m, pxy_b, pxy_beta_m, pxy_mu, pxy_delta_m) begin
    @tr begin
        @s("2U") => @s("UO"), 2 * pxy_gamma
        @s("UO") => @s("1D"), pxy_d * pxy_delta1
        @s("UO") => @s("2D"), pxy_e * pxy_delta2
        @s("UO") => @s("1N"), (1-pxy_d)*pxy_delta1
        @s("1N") => @s("2D"), pxy_e*pxy_delta2
        @s("1N") => @s("UN"), (1-pxy_e)*pxy_delta2
        @s("UO") => @s("2N"), (1-pxy_e)*pxy_delta2
        @s("2N") => @s("UN"), (1-pxy_d)*pxy_delta1
        @s("2N") => @s("1D2N"), pxy_d*pxy_delta1
        @s("1D") => @s("SO"), pxy_c * pxy_phi
        @s("1D") => @s("SN"), (1-pxy_c) * pxy_phi
        @s("SO") => @s("UA"), pxy_e * pxy_delta2
        @s("SO") => @s("UR"), (1-pxy_e) * pxy_delta2
        @s("SN") => @s("UC"), pxy_e * pxy_delta2
        @s("SN") => @s("US"), (1-pxy_e) * pxy_delta2
        @s("2D") => @s("1D2D"), pxy_delta1
        @s("1D2D") => @s("UA"), pxy_c * pxy_phi
        @s("1D2D") => @s("UC"), (1-pxy_c) * pxy_phi
        @s("UA") => @s("UR"), (1-pxy_q) * pxy_rou_a
        @s("UA") => @s("2U"), pxy_q * pxy_rou_a
        @s("UR") => @s("2U"), pxy_r * pxy_rou_m
        @s("UR") => @s("UB"), (1-pxy_r) * pxy_rou_m
        @s("UB") => @s("2U"), pxy_beta_m
        @s("UC") => @s("US"), (1-pxy_q) * pxy_rou_a
        @s("UC") => @s("2U"), pxy_q * pxy_rou_a
        @s("US") => @s("2U"), pxy_r * pxy_rou_m
        @s("US") => @s("UT"), (1-pxy_r) * pxy_rou_m
        @s("UT") => @s("2U"), pxy_b * pxy_beta_m
        @s("UT") => @s("RP"), (1-pxy_b) * pxy_beta_m
        @s("RP") => @s("2U"), pxy_mu
        @s("UN") => @s("MD"), pxy_delta_m 
        @s("MD") => @s("1D2N"), pxy_delta1
        @s("1D2N") => @s("UR"), pxy_c * pxy_phi
        @s("1D2N") => @s("US"), (1-pxy_c) * pxy_phi
    end
    
    @init begin
        @s("2U"), 1
    end
end

pxy_pairup (generic function with 1 method)

In [10]:
## *========availability model for single appserver, Figure 19========*
@markov appserver(gamma, d, delta1, e, delta2, q, rou_a, r, rou_m, b, beta_m, mu, delta_m) begin
    @tr begin
        @s("UP") => @s("UO"), gamma
        @s("UO") => @s("UA"), e*delta2
        @s("UO") => @s("1D"), d*delta1
        @s("UO") => @s("1N"), (1-d)*delta1
        @s("UO") => @s("2N"), (1-e)*delta2
        @s("1N") => @s("UA"), e*delta2
        @s("1N") => @s("UN"), (1-e)*delta2
        @s("2N") => @s("UR"), d*delta1
        @s("2N") => @s("UN"), (1-d)*delta1
        @s("1D") => @s("UA"), e*delta2
        @s("1D") => @s("UR"), (1-e)*delta2
        @s("UA") => @s("UR"), (1-q)*rou_a
        @s("UR") => @s("UB"), (1-r)*rou_m
        @s("UB") => @s("RE"), (1-b)*beta_m
        @s("UN") => @s("UR"), delta_m
        @s("UA") => @s("UP"), q*rou_a
        @s("UR") => @s("UP"), r*rou_m
        @s("UB") => @s("UP"), b*beta_m
        @s("RE") => @s("UP"), mu
    end
    
    @reward :up begin
        @s("UP"), 1
    end
end

appserver (generic function with 1 method)

In [11]:
### *========availability model for single proxy server, Figure 19========*
@markov proxy(pxy_gamma, pxy_d, pxy_delta1, pxy_e, pxy_delta2, pxy_q, pxy_rou_a, pxy_r, pxy_rou_m, pxy_b, pxy_beta_m, pxy_mu, pxy_delta_m) begin
    @tr begin
        @s("UP") => @s("UO"), pxy_gamma
        @s("UO") => @s("UA"), pxy_e*pxy_delta2
        @s("UO") => @s("1N"), (1-pxy_d)*pxy_delta1
        @s("UO") => @s("2N"), (1-pxy_e)*pxy_delta2
        @s("1N") => @s("UA"), pxy_e*pxy_delta2
        @s("1N") => @s("UN"), (1-pxy_e)*pxy_delta2
        @s("2N") => @s("UR"), pxy_d*pxy_delta1
        @s("2N") => @s("UN"), (1-pxy_d)*pxy_delta1
        @s("UA") => @s("UR"), (1-pxy_q)*pxy_rou_a
        @s("UR") => @s("UB"), (1-pxy_r)*pxy_rou_m
        @s("UB") => @s("RE"), (1-pxy_b)*pxy_beta_m
        @s("UO") => @s("1D"), pxy_d*pxy_delta1
        @s("1D") => @s("UA"), pxy_e*pxy_delta2
        @s("1D") => @s("UR"), (1-pxy_e)*pxy_delta2
        @s("UN") => @s("UR"), pxy_delta_m
        @s("UA") => @s("UP"), pxy_q*pxy_rou_a
        @s("UR") => @s("UP"), pxy_r*pxy_rou_m
        @s("UB") => @s("UP"), pxy_b*pxy_beta_m
        @s("RE") => @s("UP"), pxy_mu
    end
    
    @reward :up begin
        @s("UP"), 1
    end
end

proxy (generic function with 1 method)

In [12]:
### *=====================availability models for hardware=============================
#### *=======Figure 13==========*
@markov midplane(c_mp, lambda_mp, alpha_sp, mu_mp) begin
    @tr begin
        @s("UP") => @s("DN"), c_mp*lambda_mp
        @s("UP") => @s("U1"), (1-c_mp)*lambda_mp
        @s("U1") => @s("RP"), alpha_sp
        @s("RP") => @s("UP"), mu_mp
        @s("DN") => @s("RP"), alpha_sp
    end
    @reward :up begin
        @s("UP"), 1
        @s("U1"), 1
    end
end

#### *=======Figure 14==========*
@markov cooling(lambda_c, alpha_sp, mu_c, mu_2c) begin
    @tr begin
        @s("UP") => @s("U1"), 2*lambda_c
        @s("U1") => @s("RP"), alpha_sp
        @s("U1") => @s("DN"), lambda_c
        @s("DN") => @s("DW"), alpha_sp
        @s("RP") => @s("UP"), mu_c
        @s("RP") => @s("DW"), lambda_c
        @s("DW") => @s("UP"), mu_2c
    end
    @reward :up begin
        @s("UP"), 1
        @s("U1"), 1
        @s("RP"), 1
    end
end

#### *=======Figure 15==========*
@markov power(c_ps, lambda_ps, alpha_sp, mu_ps, mu_2ps) begin
    @tr begin
        @s("UP") => @s("U1"), 2*c_ps*lambda_ps
        @s("UP") => @s("DN"), 2*(1-c_ps)*lambda_ps
        @s("U1") => @s("RP"), alpha_sp
        @s("U1") => @s("DN"), lambda_ps
        @s("DN") => @s("DW"), alpha_sp
        @s("RP") => @s("UP"), mu_ps
        @s("RP") => @s("DW"), lambda_ps
        @s("DW") => @s("UP"), mu_2ps
    end
    @reward :up begin
        @s("UP"), 1
        @s("U1"), 1
        @s("RP"), 1
    end
end

#### *=======Figure 16==========*
@markov processor(lambda_cpu, alpha_sp, mu_cpu) begin
    @tr begin
        @s("UP") => @s("D1"), 2*lambda_cpu
        @s("D1") => @s("RP"), alpha_sp
        @s("RP") => @s("UP"), mu_cpu
    end
    @reward :up begin
        @s("UP"), 1
    end
end

#### *=======Figure 17==========*
@markov memory(lambda_mem, alpha_sp, mu_mem) begin
    @tr begin
        @s("UP") => @s("D1"), 4*lambda_mem
        @s("D1") => @s("RP"), alpha_sp
        @s("RP") => @s("UP"), mu_mem
    end
    @reward :up begin
        @s("UP"), 1
    end
end

#### *=======Figure 18==========*
@markov disk(lambda_hd, alpha_sp, mu_hd, chi_hd, mu_2hd) begin
    @tr begin
        @s("UP") => @s("U1"), 2*lambda_hd
        @s("U1") => @s("RP"), alpha_sp
        @s("RP") => @s("CP"), mu_hd
        @s("CP") => @s("DW"), lambda_hd
        @s("U1") => @s("DN"), lambda_hd
        @s("DN") => @s("DW"), alpha_sp
        @s("CP") => @s("UP"), chi_hd
        @s("DW") => @s("UP"), mu_2hd
    end
    @reward :up begin
        @s("UP"), 1
        @s("U1"), 1
        @s("CP"), 1
    end
end

#### *=======Figure 20==========*
@markov base(lambda_base, alpha_sp, mu_base) begin
    @tr begin
        @s("UP") => @s("DN"), lambda_base
        @s("DN") => @s("RP"), alpha_sp
        @s("RP") => @s("UP"), mu_base
    end
    @reward :up begin
        @s("UP"), 1
    end
end

#### *=======Figure 21==========*
@markov OS(lambda_os, delta_os, b_os, beta_os, alpha_sp, mu_os) begin
    @tr begin
        @s("UP") => @s("DN"), lambda_os
        @s("DN") => @s("DT"), delta_os
        @s("DT") => @s("UP"), b_os*beta_os
        @s("DT") => @s("DW"), (1-b_os)*beta_os
        @s("DW") => @s("RP"), alpha_sp
        @s("RP") => @s("UP"), mu_os
    end
    @reward :up begin
        @s("UP"), 1
    end
end

#### *=======Figure 20==========*
@markov nic(lambda_nic, alpha_sp, mu_nic) begin
    @tr begin
        @s("UP") => @s("DN"), lambda_nic
        @s("DN") => @s("RP"), alpha_sp
        @s("RP") => @s("UP"), mu_nic
    end
    @reward :up begin
        @s("UP"), 1
    end
end

#### *=======Figure 20==========*
@markov switch(lambda_esw, alpha_sp, mu_esw) begin
    @tr begin
        @s("UP") => @s("DN"), lambda_esw
        @s("DN") => @s("RP"), alpha_sp
        @s("RP") => @s("UP"), mu_esw
    end
    @reward :up begin
        @s("UP"), 1
    end
end

switch (generic function with 1 method)

In [13]:
#### *=======Figure 22==========*

@ftree network(t, lambda_nic, lambda_esw) begin
    @repeat begin
        nic1 = 1 - exp(-lambda_nic*t)
        nic2 = 1 - exp(-lambda_nic*t)
        esw1 = 1 - exp(-lambda_esw*t)
        esw2 = 1 - exp(-lambda_esw*t)
    end
    eth1 = nic1 | esw1
    eth2 = nic2 | esw2
    top = eth1 & eth2
end

#### *=======Figure 11==========*

@ftree CM(midplane, cooling, power) begin
    @repeat begin
        MP = 1-exrss(midplane, reward=:up)
        Cool = 1-exrss(cooling, reward=:up)
        Pwr = 1-exrss(power, reward=:up)
    end
    top = MP | Cool | Pwr
end

#### *=======part of Figure 12==========*

@ftree BLADE(base, processor, memory, disk, os) begin
    @repeat begin
        Base = 1-exrss(base, reward=:up)
        CPU = 1-exrss(processor, reward=:up)
        Mem = 1-exrss(memory, reward=:up)
        RAID = 1-exrss(disk, reward=:up)
        OS = 1-exrss(os, reward=:up)
    end
    top = Base | CPU | Mem | RAID | OS
end

BLADE (generic function with 1 method)

In [14]:
### *========system availability model=========*

#### *=======Figure 10==========*
@ftree CLUSTER(CM, switch, appserver, proxy, BLADE, nic) begin
    @repeat begin
        CM1 = prob(CM)
        CM2 = prob(CM)
        esw1 = 1-exrss(switch, reward=:up)
        esw2 = 1-exrss(switch, reward=:up)
        esw3 = 1-exrss(switch, reward=:up)
        esw4 = 1-exrss(switch, reward=:up)
    end
    @basic begin
        SW = 1-exrss(appserver, reward=:up)
        SWP = 1-exrss(proxy, reward=:up)
    end
    @repeat begin
        Node_A = prob(BLADE)
        nic1_A = 1-exrss(nic, reward=:up)
        nic2_A = 1-exrss(nic, reward=:up)
        Node_B = prob(BLADE)
        nic1_B = 1-exrss(nic, reward=:up)
        nic2_B = 1-exrss(nic, reward=:up)
        Node_C = prob(BLADE)
        nic1_C = 1-exrss(nic, reward=:up)
        nic2_C = 1-exrss(nic, reward=:up)
        Node_D = prob(BLADE)
        nic1_D = 1-exrss(nic, reward=:up)
        nic2_D = 1-exrss(nic, reward=:up)
        Node_E = prob(BLADE)
        nic1_E = 1-exrss(nic, reward=:up)
        nic2_E = 1-exrss(nic, reward=:up)
        Node_F = prob(BLADE)
        nic1_F = 1-exrss(nic, reward=:up)
        nic2_F = 1-exrss(nic, reward=:up)
        Node_G = prob(BLADE)
        nic1_G = 1-exrss(nic, reward=:up)
        nic2_G = 1-exrss(nic, reward=:up)
        Node_H = prob(BLADE)
        nic1_H = 1-exrss(nic, reward=:up)
        nic2_H = 1-exrss(nic, reward=:up)
    end

    ###  *===========Blade Servers===========*
    eth1_A = nic1_A | esw1
    eth2_A = nic2_A | esw2
    eth_A = eth1_A & eth2_A
    BS_A = Node_A | eth_A

    eth1_B = nic1_B | esw1
    eth2_B = nic2_B | esw2
    eth_B = eth1_B & eth2_B
    BS_B = Node_B | eth_B

    eth1_C = nic1_C | esw1
    eth2_C = nic2_C | esw2
    eth_C = eth1_C & eth2_C
    BS_C = Node_C | eth_C

    eth1_D = nic1_D | esw3
    eth2_D = nic2_D | esw4
    eth_D = eth1_D & eth2_D
    BS_D = Node_D | eth_D

    eth1_E = nic1_E | esw3
    eth2_E = nic2_E | esw4
    eth_E = eth1_E & eth2_E
    BS_E = Node_E | eth_E

    eth1_F = nic1_F | esw3
    eth2_F = nic2_F | esw4
    eth_F = eth1_F & eth2_F
    BS_F = Node_F | eth_F

    eth1_G = nic1_G | esw1
    eth2_G = nic2_G | esw2
    eth_G = eth1_G & eth2_G
    BS_G = Node_G | eth_G

    eth1_H = nic1_H | esw3
    eth2_H = nic2_H | esw4
    eth_H = eth1_H & eth2_H
    BS_H = Node_H | eth_H

    ### *===========Application Servers===========*
    AS1 = SW | BS_A | CM1
    AS2 = SW | BS_A | CM1
    AS3 = SW | BS_B | CM1
    AS4 = SW | BS_B | CM1
    AS5 = SW | BS_C | CM1
    AS6 = SW | BS_C | CM1
    AS7 = SW | BS_D | CM2
    AS8 = SW | BS_D | CM2
    AS9 = SW | BS_E | CM2
    AS10 = SW | BS_E | CM2
    AS11 = SW | BS_F | CM2
    AS12 = SW | BS_F | CM2
    apps = kofn(6, AS1, AS2, AS3, AS4, AS5, AS6, AS7, AS8, AS9, AS10, AS11, AS12)

    ### *===========Proxy Servers===========*
    PX1 = SWP | BS_G | CM1
    PX2 = SWP | BS_H | CM2
    pxys = PX1 & PX2

    top = apps | pxys
end

CLUSTER (generic function with 1 method)

In [15]:
# bind    
#     f_esw         prob(switch, UP)*lambda_esw
#     f_nic         prob(nic, UP)*lambda_nic
#     u_esw         f_esw/(1-exrss(switch))
#     u_nic         f_nic/(1-exrss(nic))
# end  

In [16]:
env = SymbolicEnv()
@bind env begin
    b_os = 0.95
    rou_a = 41.0390478956483
    delta_m = 2
    pxy_rou_a = 76.1791903844933
    rou_m = 10.9849424929091
    mu_esw = 2
    phi = 1241.37931034483
    mo_new = 3
    mu_ps = 2
    pxy_rou_m = 12.5323316751202
    n_i = 3
    delta_os = 2
    lambda_esw = 4.07871429136622e-007
    pxy_beta_m = 5.38628248508112
    mu_base = 2
    n_o = 4
    mu_nic = 2
    pxy_delta1 = 3600
    pxy_delta2 = 5.24781341107872
    lambda_c = 2.0547943798086e-007
    mu_mp = 2
    lambda_nic = 6.43835819290738e-007
    gamma_die = 0.000342465753424658
    m_stable1 = 4
    m_stable2 = 4
    m_setup = 4
    beta_m = 5.40946656649136
    pxy_gamma_die = 0.000342465753424658
    beta_os = 6
    lambda_os = 0.000114155251141553
    lambda_hd = 1.2214605736956e-006
    pxy_gamma_hang = 0.000342465753424658
    mu_c = 2
    delta1 = 356.282784139751
    delta2 = 5.24781341107872
    lambda = 100
    mu_mem = 2
    mu_2c = 1.71428571428571
    t_setup = 0.00416666666666667
    lambda_base = 3.49314470351934e-006
    lambda_mem = 1.12442935210381e-006
    mu_2hd = 1.71428571428571
    pxy_delta_m = 2
    gamma_hang = 0.000342465753424658
    pxy_phi = 360
    pxy_mu = 0.25
    lambda_ps = 2.46575072246496e-006
    chi_hd = 12
    rw_i = 0.00111111111111111
    mu_cpu = 2
    b = 0.95
    c = 0.95
    rw_o = 0.00333333333333333
    d = 0.95
    e = 0.95
    c_ps = 0.99
    pxy_b = 0.95
    pxy_c = 0.95
    lambda_cpu = 8.10502162825022e-007
    mu_2ps = 1.71428571428571
    pxy_d = 0.95
    pxy_e = 0.95
    t_stable1 = 0.0125
    t_stable2 = 0.0166666666666667
    mu_os = 2
    q = 0.95
    alpha_sp = 0.4
    mi_new = 2
    r = 0.95
    mu_hd = 2
    lambda_mp = 1.60958799376124e-006
    mu = 0.25
    pxy_q = 0.95
    pxy_r = 0.95
    c_mp = 0.0036101083032491
end

(:b_os, :rou_a, :delta_m, :pxy_rou_a, :rou_m, :mu_esw, :phi, :mo_new, :mu_ps, :pxy_rou_m, :n_i, :delta_os, :lambda_esw, :pxy_beta_m, :mu_base, :n_o, :mu_nic, :pxy_delta1, :pxy_delta2, :lambda_c, :mu_mp, :lambda_nic, :gamma_die, :m_stable1, :m_stable2, :m_setup, :beta_m, :pxy_gamma_die, :beta_os, :lambda_os, :lambda_hd, :pxy_gamma_hang, :mu_c, :delta1, :delta2, :lambda, :mu_mem, :mu_2c, :t_setup, :lambda_base, :lambda_mem, :mu_2hd, :pxy_delta_m, :gamma_hang, :pxy_phi, :pxy_mu, :lambda_ps, :chi_hd, :rw_i, :mu_cpu, :b, :c, :rw_o, :d, :e, :c_ps, :pxy_b, :pxy_c, :lambda_cpu, :mu_2ps, :pxy_d, :pxy_e, :t_stable1, :t_stable2, :mu_os, :q, :alpha_sp, :mi_new, :r, :mu_hd, :lambda_mp, :mu, :pxy_q, :pxy_r, :c_mp)

In [17]:
gamma = gamma_hang + gamma_die
pxy_gamma = pxy_gamma_hang + pxy_gamma_die

:(pxy_gamma_hang + pxy_gamma_die)

In [18]:
nic_model = ctmc(nic(lambda_nic, alpha_sp, mu_nic), :DenseCTMC)
tmp = 1 - exrss(nic_model, reward=:up)
seval(tmp, env)

1.931503727070627e-6

In [19]:
base_model = ctmc(base(lambda_base, alpha_sp, mu_base), :DenseCTMC)
processor_model = ctmc(processor(lambda_cpu, alpha_sp, mu_cpu), :DenseCTMC)
memory_model = ctmc(memory(lambda_mem, alpha_sp, mu_mem), :DenseCTMC)
disk_model = ctmc(disk(lambda_hd, alpha_sp, mu_hd, chi_hd, mu_2hd), :DenseCTMC)
os_model = ctmc(OS(lambda_os, delta_os, b_os, beta_os, alpha_sp, mu_os), :DenseCTMC)
blade_model = BLADE(base_model, processor_model, memory_model, disk_model, os_model)
seval(prob(blade_model), env)

0.00012327175758095066

In [20]:
midplane_model = ctmc(midplane(c_mp, lambda_mp, alpha_sp, mu_mp), :DenseCTMC)
cooling_model = ctmc(cooling(lambda_c, alpha_sp, mu_c, mu_2c), :DenseCTMC)
power_model = ctmc(power(c_ps, lambda_ps, alpha_sp, mu_ps, mu_2ps), :DenseCTMC)
cm_model = CM(midplane_model, cooling_model, power_model)
seval(prob(cm_model), env)

9.714662409148886e-7

In [21]:
switch_model = ctmc(switch(lambda_esw, alpha_sp, mu_esw), :DenseCTMC)
appserver_model = ctmc(appserver(gamma, d, delta1, e, delta2, q, rou_a, r, rou_m, b, beta_m, mu, delta_m), :DenseCTMC)
proxy_model = ctmc(proxy(pxy_gamma, pxy_d, pxy_delta1, pxy_e, pxy_delta2, pxy_q, pxy_rou_a, pxy_r, pxy_rou_m, pxy_b, pxy_beta_m, pxy_mu, pxy_delta_m), :DenseCTMC)
cluster_model = CLUSTER(cm_model, switch_model, appserver_model, proxy_model, blade_model, nic_model)
@time seval(prob(cluster_model), env)

  0.063881 seconds (208.31 k allocations: 10.455 MiB, 93.89% compilation time)


2.015744522149801e-6

In [22]:
params = sort(collect(prob(cluster_model).params))

58-element Vector{Symbol}:
 :alpha_sp
 :b
 :b_os
 :beta_m
 :beta_os
 :c_mp
 :c_ps
 :chi_hd
 :d
 :delta1
 :delta2
 :delta_m
 :delta_os
 ⋮
 :pxy_e
 :pxy_gamma_die
 :pxy_gamma_hang
 :pxy_mu
 :pxy_q
 :pxy_r
 :pxy_rou_a
 :pxy_rou_m
 :q
 :r
 :rou_a
 :rou_m

In [23]:
@time [seval(prob(cluster_model), x, env) for x = params]

  2.382945 seconds (15.23 M allocations: 549.320 MiB, 6.93% gc time, 8.85% compilation time)


58-element Vector{Float64}:
 -7.416243507135751e-7
 -2.633712880138427e-14
 -1.8501523253496662e-7
 -2.250088264086389e-16
 -1.7131040049579815e-9
  8.047891478653e-6
 -3.0410224192662057e-5
  4.6649962035685925e-18
 -3.376554975086383e-14
 -1.5332534946067026e-20
 -4.904248533733392e-14
 -8.441387437711936e-16
 -1.541793604462187e-8
  ⋮
 -3.9153361167945e-8
  0.00011537619059292323
  0.00011537619059292323
 -1.4406013511893682e-9
 -3.476748364160481e-8
 -1.3889442873725756e-8
 -6.046862737577157e-11
 -2.2930845375438514e-10
 -1.4149510522919186e-13
 -5.0680683276880113e-14
 -7.618384161785597e-16
 -1.0912967370581874e-15

In [24]:
@time [seval(prob(cluster_model), (x,y), env) for x = params, y = params]

 95.780540 seconds (1.74 G allocations: 43.308 GiB, 6.86% gc time, 0.87% compilation time)


58×58 Matrix{Float64}:
  3.72846e-6    1.85499e-6    1.92099e-6   …   1.85499e-6    1.85499e-6
  4.08722e-14   2.27751e-15   1.46183e-13      6.55386e-17   9.4033e-17
  4.51389e-7   -6.32093e-11   2.36004e-7      -6.33512e-11  -6.33494e-11
  3.90787e-16   6.10561e-17   1.2905e-15       4.21583e-17   4.24017e-17
  8.96658e-10   2.85513e-10   2.47132e-9       2.85512e-10   2.85512e-10
 -2.01197e-5    2.11959e-19   1.48899e-12  …   6.13122e-21   8.78267e-21
  6.16419e-5   -1.24971e-11  -1.81234e-11     -1.24971e-11  -1.24971e-11
 -2.05367e-18  -3.88753e-19  -6.34097e-18     -3.8875e-19   -3.8875e-19
  5.24002e-14   2.91976e-15   1.87414e-13      8.38967e-17   1.20428e-16
  2.38586e-20   1.39017e-21   8.51669e-20      1.02444e-22   1.19032e-22
  8.54532e-14   1.35858e-14   2.81553e-13  …   9.46689e-15   9.51995e-15
  1.73209e-15   4.95077e-16   5.10744e-15      4.24181e-16   4.25094e-16
  1.32088e-8    7.70854e-9    2.73808e-8       7.70853e-9    7.70853e-9
  ⋮                             